<a href="https://colab.research.google.com/github/Extralit/papers-ocr-benchmarks/blob/main/marker_demo_blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marker Demo: Research Paper Block Extraction & Exploration

Self-contained, Google Colab-ready.

- Upload a PDF, run Marker to extract JSON structure
- Flatten and filter block structure (inspired by TypeScript)
- Explore block types, metadata, tables, and figures
- No LLM-based summarization or captioning included


In [38]:
# 1. Install marker-pdf and dependencies
!uv pip install --quiet marker-pdf[full] docling
!uv pip install -q "mineru[all]"
!uv pip install -q "PyMuPDF>=1.23.0" "pandas>=1.5.0" pymupdf4llm llama_index
!uv pip install -q "matplotlib>=3.5.0" "seaborn>=0.11.0" "textdistance>=4.6.0"

In [3]:
# prompt: import HTML and display from ipython

from IPython.display import HTML, display, JSON
from pprint import pprint

In [4]:
# 2. Upload a PDF
from google.colab import files
uploaded = files.upload()
pdf_path = next(iter(uploaded))

Saving Allossogbe_et_al_2017_Mal_J.pdf to Allossogbe_et_al_2017_Mal_J.pdf


In [5]:
file_path = "/content/Allossogbe_et_al_2017_Mal_J.pdf"

## MinerU

In [29]:
%%time
!mineru -p /content/Allossogbe_et_al_2017_Mal_J.pdf -o /content/mineru_output/

2025-07-08 20:03:55.332 | INFO     | mineru.backend.pipeline.pipeline_analyze:doc_analyze:124 - Batch 1/1: 11 pages/11 pages
2025-07-08 20:03:55.334 | INFO     | mineru.backend.pipeline.pipeline_analyze:batch_image_analyze:187 - gpu_memory: 15 GB, batch_ratio: 8
2025-07-08 20:03:55.334 | INFO     | mineru.backend.pipeline.model_init:__init__:137 - DocAnalysis init, this may take some times......
2025-07-08 20:04:09.188 | INFO     | mineru.backend.pipeline.model_init:__init__:182 - DocAnalysis init done!
2025-07-08 20:04:09.189 | INFO     | mineru.backend.pipeline.pipeline_analyze:custom_model_init:64 - model init cost: 13.854581832885742
Layout Predict: 100% 11/11 [00:02<00:00,  4.03it/s]
MFD Predict: 100% 11/11 [00:04<00:00,  2.23it/s]
MFR Predict: 100% 140/140 [00:06<00:00, 21.88it/s]
OCR-det ch: 100% 65/65 [00:09<00:00,  7.19it/s]
Table Predict: 100% 4/4 [00:07<00:00,  1.76s/it]
Processing pages: 100% 11/11 [00:09<00:00,  1.15it/s]
OCR-rec Predict: 100% 5/5 [00:00<00:00, 44.46it/s]


## Docling

In [48]:
%%time
from docling.document_converter import DocumentConverter
converter = DocumentConverter()
result = converter.convert(file_path)

CPU times: user 1min 14s, sys: 3.18 s, total: 1min 17s
Wall time: 1min 13s


In [ ]:
print(result.document.export_to_markdown())

## PyMuPDF4LLM

Read https://pymupdf.readthedocs.io/en/latest/pymupdf4llm/api.html#pymupdf4llm-api

In [49]:
%%time
import pymupdf4llm
md_text = pymupdf4llm.to_markdown(file_path, table_strategy=None)

CPU times: user 5.22 s, sys: 190 ms, total: 5.41 s
Wall time: 6.42 s


In [36]:
print(md_text)

Allossogbe et al. Malar J (2017) 16:77
DOI 10.1186/s12936-017-1727-x

### **RESEARCH**


## Malaria Journal

### **Open Access**


# WHO cone bio‑assays of classical and new‑generation long‑lasting insecticidal nets call for innovative insecticides targeting the knock‑down resistance mechanism in Benin

Marius Allossogbe [1,2*], Virgile Gnanguenon [1,2], Boulais Yovogan [1,2], Bruno Akinro [1], Rodrigue Anagonou [1,2],
Fiacre Agossa [1,2], André Houtoukpe [3], Germain Gil Padonou [1,2] and Martin Akogbeto [1,2]


**Abstract**

**Background:** To increase the effectiveness of insecticide-treated nets (ITN) in areas of high resistance, new longlasting insecticidal nets (LLINs) called new-generation nets have been developed. These nets are treated with the
piperonyl butoxide (PBO) synergist which inhibit the action of detoxification enzymes. The effectiveness of the
new-generation nets has been proven in some studies, but their specific effect on mosquitoes carrying detoxification enzymes

In [39]:
%%time
llama_reader = pymupdf4llm.LlamaMarkdownReader()
llama_docs = llama_reader.load_data(file_path)

Successfully imported LlamaIndex
CPU times: user 16 s, sys: 631 ms, total: 16.6 s
Wall time: 17.5 s


In [47]:
[(display(p.metadata), print(p.text)) for p in llama_docs]

{'format': 'PDF 1.7',
 'title': 'WHO cone bio-assays of classical and new-generation long-lasting insecticidal nets call for innovative insecticides targeting the knock-down resistance mechanism in Benin',
 'author': 'Marius Allossogbe',
 'subject': 'Malaria Journal, doi:10.1186/s12936-017-1727-x',
 'keywords': 'LLINs,Bio-efficacy,Piperonyl butoxide,Resistant mosquitoes',
 'creator': 'ocrmypdf 16.1.2 / Tesseract OCRhOCR 5.3.4',
 'producer': 'pikepdf 8.13.0',
 'creationDate': "D:20170214204129+05'30'",
 'modDate': "D:20240423063541+00'00'",
 'trapped': '',
 'encryption': None,
 'page': 1,
 'total_pages': 11,
 'file_path': '/content/Allossogbe_et_al_2017_Mal_J.pdf'}

Allossogbe et al. Malar J (2017) 16:77
DOI 10.1186/s12936-017-1727-x

### **RESEARCH**


## Malaria Journal

### **Open Access**


# WHO cone bio‑assays of classical and new‑generation long‑lasting insecticidal nets call for innovative insecticides targeting the knock‑down resistance mechanism in Benin

Marius Allossogbe [1,2*], Virgile Gnanguenon [1,2], Boulais Yovogan [1,2], Bruno Akinro [1], Rodrigue Anagonou [1,2],
Fiacre Agossa [1,2], André Houtoukpe [3], Germain Gil Padonou [1,2] and Martin Akogbeto [1,2]


**Abstract**

**Background:** To increase the effectiveness of insecticide-treated nets (ITN) in areas of high resistance, new longlasting insecticidal nets (LLINs) called new-generation nets have been developed. These nets are treated with the
piperonyl butoxide (PBO) synergist which inhibit the action of detoxification enzymes. The effectiveness of the
new-generation nets has been proven in some studies, but their specific effect on mosquitoes carrying detoxification enzymes

{'format': 'PDF 1.7',
 'title': 'WHO cone bio-assays of classical and new-generation long-lasting insecticidal nets call for innovative insecticides targeting the knock-down resistance mechanism in Benin',
 'author': 'Marius Allossogbe',
 'subject': 'Malaria Journal, doi:10.1186/s12936-017-1727-x',
 'keywords': 'LLINs,Bio-efficacy,Piperonyl butoxide,Resistant mosquitoes',
 'creator': 'ocrmypdf 16.1.2 / Tesseract OCRhOCR 5.3.4',
 'producer': 'pikepdf 8.13.0',
 'creationDate': "D:20170214204129+05'30'",
 'modDate': "D:20240423063541+00'00'",
 'trapped': '',
 'encryption': None,
 'page': 2,
 'total_pages': 11,
 'file_path': '/content/Allossogbe_et_al_2017_Mal_J.pdf'}

Allossogbe et al. Malar J (2017) 16:77 Page 2 of 11



**Background**
Malaria is a major public health problem worldwide,
and particularly so in Benin. It remains a permanent
threat from its high morbidity (214 million) and mortality (438,000). Africa is the most endemic region
affected (395,000 deaths per year) [1]. It affects onefifth of the world population. However, this proportion
has decreased significantly by 37% between 2000 and
2015 due to the effect of malaria prevention and treatment methods, including long-lasting insecticidal nets
(LLINs), indoor residual spraying of residual insecticides
(IRS), chemo-prevention for pregnant women and children, and therapeutic treatment with artemisinin-based
combinations.

Among these prevention methods, LLINs have
emerged in recent years as a privileged tool to prevent
malaria. Te insecticides selected by the World Health
Organization (WHO) for LLIN treatment are pyrethroids, which have little toxicity to humans, are effective at low dos

{'format': 'PDF 1.7',
 'title': 'WHO cone bio-assays of classical and new-generation long-lasting insecticidal nets call for innovative insecticides targeting the knock-down resistance mechanism in Benin',
 'author': 'Marius Allossogbe',
 'subject': 'Malaria Journal, doi:10.1186/s12936-017-1727-x',
 'keywords': 'LLINs,Bio-efficacy,Piperonyl butoxide,Resistant mosquitoes',
 'creator': 'ocrmypdf 16.1.2 / Tesseract OCRhOCR 5.3.4',
 'producer': 'pikepdf 8.13.0',
 'creationDate': "D:20170214204129+05'30'",
 'modDate': "D:20240423063541+00'00'",
 'trapped': '',
 'encryption': None,
 'page': 3,
 'total_pages': 11,
 'file_path': '/content/Allossogbe_et_al_2017_Mal_J.pdf'}

...

{'format': 'PDF 1.7',
 'title': 'WHO cone bio-assays of classical and new-generation long-lasting insecticidal nets call for innovative insecticides targeting the knock-down resistance mechanism in Benin',
 'author': 'Marius Allossogbe',
 'subject': 'Malaria Journal, doi:10.1186/s12936-017-1727-x',
 'keywords': 'LLINs,Bio-efficacy,Piperonyl butoxide,Resistant mosquitoes',
 'creator': 'ocrmypdf 16.1.2 / Tesseract OCRhOCR 5.3.4',
 'producer': 'pikepdf 8.13.0',
 'creationDate': "D:20170214204129+05'30'",
 'modDate': "D:20240423063541+00'00'",
 'trapped': '',
 'encryption': None,
 'page': 10,
 'total_pages': 11,
 'file_path': '/content/Allossogbe_et_al_2017_Mal_J.pdf'}

Allossogbe et al. Malar J (2017) 16:77 Page 10 of 11



**Author details**
1 Centre de Recherche Entomologique de Cotonou (CREC), Cotonou,
Benin. [2] Université d’Abomey-Calavi, Abomey‑Calavi, Benin. [3] Medical Care
and Development International, Washington, USA.


**Acknowledgements**
We thank CREC personnel for their technical assistance and collaboration.


**Competing interests**
The authors declare that they have no competing interests.


**Availability of data and materials**
Data collected during this study are included in the published article and its
additional files.


**Funding**
This work is supported by Faculty of Letters, Arts and Human Sciences of the
University of Abomey-Calavi.


Received: 5 December 2016  Accepted: 7 February 2017


**References**

1. WHO. World malaria report 2015. Geneva: World Health Organiza[tion; 2015. http://www.who.int/malaria/publications/world-malaria-](http://www.who.int/malaria/publications/world-malaria-report-2015/report/en/)
[report-201

{'format': 'PDF 1.7',
 'title': 'WHO cone bio-assays of classical and new-generation long-lasting insecticidal nets call for innovative insecticides targeting the knock-down resistance mechanism in Benin',
 'author': 'Marius Allossogbe',
 'subject': 'Malaria Journal, doi:10.1186/s12936-017-1727-x',
 'keywords': 'LLINs,Bio-efficacy,Piperonyl butoxide,Resistant mosquitoes',
 'creator': 'ocrmypdf 16.1.2 / Tesseract OCRhOCR 5.3.4',
 'producer': 'pikepdf 8.13.0',
 'creationDate': "D:20170214204129+05'30'",
 'modDate': "D:20240423063541+00'00'",
 'trapped': '',
 'encryption': None,
 'page': 11,
 'total_pages': 11,
 'file_path': '/content/Allossogbe_et_al_2017_Mal_J.pdf'}

Allossogbe et al. Malar J (2017) 16:77 Page 11 of 11



35. Yewhalaw D, Asale A, Tushune K, Getachew Y, Duchateau L, Speybroeck
N. Bio-efficacy of selected long-lasting insecticidal nets against pyrethroid resistant Anopheles arabiensis from South-Western Ethiopia. Parasit
Vectors. 2012;5:159.
36. Pennetier C, Bouraima A, Chandre F, Piameu M, Etang J, Rossignol M, et al.
Efficacy of Olyset [®] Plus, a new long-lasting insecticidal net incorporating
permethrin and piperonil-butoxide against multi-resistant malaria vectors. PLoS ONE. 2013;8:e75134.
37. Ranson H, Abdallah H, Badolo A, Guelbeogo WM, Kerah-Hinzoumbé C,
Yangalbé-Kalnoné E, et al. Insecticide resistance in Anopheles gambiae:
data from the first year of a multi-country study highlight the extent of
the problem. Malar J. 2009;8:299.
38. Chouaibou MS, Chabi J, Bingham GV, Knox TB, N’Dri L, Kesse NB, et al.
Increase in susceptibility to insecticides with aging of wild Anopheles
gambiae mosquitoes from Côte d’Ivoire. BMC Infect Di

[(None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None)]

## Marker

In [7]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.schema import BlockTypes

artifact_dict = create_model_dict()
artifact_dict

{'layout_model': <surya.layout.LayoutPredictor at 0x7f93a32e4590>,
 'recognition_model': <surya.recognition.RecognitionPredictor at 0x7f93a428e110>,
 'table_rec_model': <surya.table_rec.TableRecPredictor at 0x7f93a2f84410>,
 'detection_model': <surya.detection.DetectionPredictor at 0x7f93a2eb9710>,
 'ocr_error_model': <surya.ocr_error.OCRErrorPredictor at 0x7f93a32d7e90>}

In [ ]:
converter = PdfConverter(
    artifact_dict=create_model_dict(),
)

document = converter.build_document(file_path)
forms = document.contained_blocks((BlockTypes.Form,))

Running OCR Error Detection: 100%|██████████| 1/1 [00:00<00:00, 26.08it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:01<00:00,  1.26s/it]


In [ ]:
blocks = document.contained_blocks((BlockTypes.Text,))
blocks[0]

Text(polygon=PolygonBox(polygon=[[56.45703125, 229.614013671875], [506.076171875, 229.614013671875], [506.076171875, 257.422607421875], [56.45703125, 257.422607421875]], bbox=[56.45703125, 229.614013671875, 506.076171875, 257.422607421875]), block_description='A paragraph or line of text.', block_type=<BlockTypes.Text: '23'>, block_id=6, page_id=0, text_extraction_method='pdftext', structure=[/page/0/Line/40, /page/0/Line/58, /page/0/Line/64, /page/0/Line/67, /page/0/Line/77], ignore_for_output=False, replace_output_newlines=False, source='layout', top_k={<BlockTypes.Text: '23'>: 0.9404296875, <BlockTypes.TextInlineMath: '16'>: 0.05963134765625, <BlockTypes.Code: '10'>: 1.1980533599853516e-05, <BlockTypes.Footnote: '12'>: 4.947185516357422e-06, <BlockTypes.Form: '13'>: 4.827976226806641e-06}, metadata=None, lowres_image=None, highres_image=None, removed=False, has_continuation=False, blockquote=False, blockquote_level=0, html=None)

In [ ]:
blocks[0].html

In [ ]:
pprint(blocks)

[Text(polygon=PolygonBox(polygon=[[56.45703125, 229.614013671875], [506.076171875, 229.614013671875], [506.076171875, 257.422607421875], [56.45703125, 257.422607421875]], bbox=[56.45703125, 229.614013671875, 506.076171875, 257.422607421875]), block_description='A paragraph or line of text.', block_type=<BlockTypes.Text: '23'>, block_id=6, page_id=0, text_extraction_method='pdftext', structure=[/page/0/Line/40, /page/0/Line/58, /page/0/Line/64, /page/0/Line/67, /page/0/Line/77], ignore_for_output=False, replace_output_newlines=False, source='layout', top_k={<BlockTypes.Text: '23'>: 0.9404296875, <BlockTypes.TextInlineMath: '16'>: 0.05963134765625, <BlockTypes.Code: '10'>: 1.1980533599853516e-05, <BlockTypes.Footnote: '12'>: 4.947185516357422e-06, <BlockTypes.Form: '13'>: 4.827976226806641e-06}, metadata=None, lowres_image=None, highres_image=None, removed=False, has_continuation=False, blockquote=False, blockquote_level=0, html=None),
 Text(polygon=PolygonBox(polygon=[[62.27734375, 303.

In [ ]:
# 3. Run Marker to extract JSON structure
import os
output_dir = 'marker_output'
os.makedirs(output_dir, exist_ok=True)
json_out = os.path.join(output_dir, os.path.splitext(os.path.basename(pdf_path))[0] + '_structure.json')

!marker_single "{pdf_path}" --output_format json --output_dir "{output_dir}"

In [ ]:
# 4. Load the Marker JSON
import json
with open(json_out, 'r') as f:
    marker_json = json.load(f)

In [ ]:
# 5. Data models and flattening utilities (Python version of your TypeScript)
from typing import List, Dict, Any

class SimplifiedBlock:
    def __init__(self, type: str, content: str, page: int, bbox: list):
        self.type = type
        self.content = content
        self.page = page
        self.bbox = bbox

    def as_dict(self):
        return {
            'type': self.type,
            'content': self.content,
            'page': self.page,
            'bbox': self.bbox,
        }

import html

def decode_html_entities(text: str) -> str:
    return html.unescape(text)

def flatten_marker_json(blocks: List[Dict[str, Any]], page_number: int = 0) -> List[SimplifiedBlock]:
    flat_blocks = []
    for block in blocks:
        # Skip Page blocks but process their children
        if block.get('block_type') == 'Page':
            child_page = int(block.get('id', '0/0/0').split('/')[2]) if 'id' in block else 0
            flat_blocks.extend(flatten_marker_json(block.get('children', []), child_page))
            continue

        # Process current block
        content = ''
        if block.get('images') and isinstance(block['images'], dict) and block['images']:
            content = next(iter(block['images'].values()))
        elif block.get('block_type') == 'Table':
            content = block.get('html', '').strip()
        elif block.get('html'):
            import re
            content = re.sub(r'<[^>]*>', ' ', block['html']).strip()
        content = decode_html_entities(content)

        page = (int(block.get('id', '0/0/0').split('/')[2]) if 'id' in block else page_number) + 1
        bbox = block.get('bbox', [0,0,0,0])

        flat_blocks.append(SimplifiedBlock(
            type=block.get('block_type', ''),
            content=content,
            page=page,
            bbox=bbox
        ))

        # Recursively process children (except for Page blocks)
        if block.get('children'):
            flat_blocks.extend(flatten_marker_json(block['children'], page))
    return flat_blocks

def filter_and_flatten_marker_json(blocks: List[Dict[str, Any]], page_number: int = 0) -> List[SimplifiedBlock]:
    unfiltered = flatten_marker_json(blocks, page_number)
    remove_types = {
        'TableCell', 'TableGroup', 'FigureGroup', 'ListGroup', 'Reference',
        'PageFooter', 'PageHeader', 'Footnote'
    }
    return [b for b in unfiltered if b.type not in remove_types and b.content]

In [ ]:
# 6. Flatten and filter the Marker output
flat_blocks = filter_and_flatten_marker_json(marker_json.get('children', []))

In [ ]:
# 7. Explore block types and content
import pandas as pd

df = pd.DataFrame([b.as_dict() for b in flat_blocks])
print('Block types found:', df['type'].unique())
df.head(20)  # Show first 20 blocks

In [ ]:
# 8. Simple metadata extraction (title, authors, abstract)
def extract_metadata(blocks: List[SimplifiedBlock]):
    title = next((b.content for b in blocks if b.type.lower() in {'title', 'main_title'}), '')
    authors = next((b.content for b in blocks if 'author' in b.type.lower()), '')
    abstract = next((b.content for b in blocks if 'abstract' in b.type.lower()), '')
    return {'title': title, 'authors': authors, 'abstract': abstract}

metadata = extract_metadata(flat_blocks)
print('Extracted Metadata:', metadata)

In [ ]:
# 9. Find and display all tables and figures (with extensibility for custom processing)
tables = [b for b in flat_blocks if b.type == 'Table']
figures = [b for b in flat_blocks if b.type == 'Figure' or b.type == 'Picture']

print(f'Found {len(tables)} tables and {len(figures)} figures.')

# Example: Show first table's HTML (for further processing)
if tables:
    from IPython.display import display, HTML
    print('First table HTML:')
    display(HTML(tables[0].content))

# Example: Show first figure as image (if base64-encoded)
import base64
from IPython.display import Image

def show_base64_image(b64str):
    try:
        display(Image(data=base64.b64decode(b64str)))
    except Exception as e:
        print('Could not display image:', e)

if figures:
    print('First figure (if image):')
    show_base64_image(figures[0].content)

In [ ]:
# 10. (Optional) Extensible: Add your own logic to process tables/figures, e.g., send table HTML to a model, extract captions, etc.
# (No LLM-based summarization or captioning included)

In [ ]:
# 11. Save flattened blocks for further analysis
df.to_json('flattened_blocks.json', orient='records', indent=2)
from google.colab import files
files.download('flattened_blocks.json')